In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

Load Images

In [2]:
import os
from PIL import Image
import numpy as np

parent_directory = "/content/drive/MyDrive/datasets/brain_tumor_dataset"
X = []
y = []

# Define class labels \
class_labels = {'glioma': 0, 'healthy': 1, 'meningioma': 2, 'pituitary': 3}

for dir_name in os.listdir(parent_directory):
    dir_path = os.path.join(parent_directory, dir_name)

    if os.path.isdir(dir_path) and dir_name in class_labels:
        print(f"Directory: {dir_name}")

        for file_name in os.listdir(dir_path):
            file_path = os.path.join(dir_path, file_name)

            if os.path.isfile(file_path):
                try:
                    img = Image.open(file_path)

                    # Convert the image to grayscale (L mode)
                    img = img.convert('L')

                    # Resize the image to a standard size (e.g., 150x150) to make it uniform
                    img = img.resize((150, 150))

                    img_array = np.array(img)
                    X.append(img_array)
                    y.append(class_labels[dir_name])
                except Exception as e:
                    print(f"Error loading image {file_name}: {e}")

# Convert lists to numpy arrays
X = np.array(X)
y = np.array(y)

print(f"Loaded {len(X)} images with {len(y)} labels.")


Directory: healthy
Directory: meningioma
Directory: pituitary
Directory: glioma
Loaded 7023 images with 7023 labels.


Perform data augmentation

In [3]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)
print(f"Training set size: {len(X_train)} images")
print(f"Test set size: {len(X_test)} images")

# 2. Data Augmentation for Training Set
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    brightness_range=(0.3, 1.0),
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow(
    X_train,
    y_train,
    batch_size=32,
    subset='training'
)

# Create the generator for the validation set (from training data)
validation_generator = train_datagen.flow(
    X_train,
    y_train,
    batch_size=32,
    subset='validation'
)

# For the actual test set (untouched during training), you can use test_datagen
test_generator = test_datagen.flow(
    X_test,
    y_test,
    batch_size=32
)

Training set size: 5618 images
Test set size: 1405 images


Build and train model

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
# Define the model
model = Sequential()

# First convolution layer
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Second convolution layer
model.add(Conv2D(64, (3, 3)))  # Increased the number of filters to 64
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Third convolution layer
model.add(Conv2D(128, (3, 3)))  # Further increased to 128 filters
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Fourth convolution layer (Newly added)
model.add(Conv2D(256, (3, 3)))  # Adding more filters to capture deeper features
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Fifth convolution layer (Newly added)
model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output
model.add(Flatten())

# Fully connected layer 1
model.add(Dense(256))  # Increased the number of units for more complexity
model.add(Activation('relu'))

# Dropout to prevent overfitting
model.add(Dropout(0.5))  # Increased dropout rate slightly to prevent overfitting with deeper layers

# Fully connected layer 2
model.add(Dense(128))
model.add(Activation('relu'))

# Dropout to prevent overfitting
model.add(Dropout(0.5))

# Fully connected layer 3
model.add(Dense(64))
model.add(Activation('relu'))

# Dropout to prevent overfitting
model.add(Dropout(0.5))

# Output layer for multiclass classification
model.add(Dense(5))  # 5 output units for 5 classes
model.add(Activation('softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_generator, epochs=50, validation_data=validation_generator, callbacks=[early_stopping])




Epoch 1/50
141/141 ━━━━━━━━━━━━━━━━━━━━ 27s 147ms/step - accuracy: 0.2612 - loss: 1.4966 - val_accuracy: 0.3170 - val_loss: 1.3113
Epoch 2/50
141/141 ━━━━━━━━━━━━━━━━━━━━ 31s 98ms/step - accuracy: 0.3504 - loss: 1.3316 - val_accuracy: 0.3767 - val_loss: 1.2821
Epoch 3/50
141/141 ━━━━━━━━━━━━━━━━━━━━ 20s 97ms/step - accuracy: 0.3760 - loss: 1.2623 - val_accuracy: 0.4390 - val_loss: 1.1480
Epoch 4/50
141/141 ━━━━━━━━━━━━━━━━━━━━ 15s 100ms/step - accuracy: 0.4198 - loss: 1.1713 - val_accuracy: 0.4524 - val_loss: 1.1206
Epoch 5/50
141/141 ━━━━━━━━━━━━━━━━━━━━ 15s 107ms/step - accuracy: 0.4426 - loss: 1.1273 - val_accuracy: 0.4782 - val_loss: 1.0612
Epoch 6/50
141/141 ━━━━━━━━━━━━━━━━━━━━ 15s 98ms/step - accuracy: 0.4697 - loss: 1.0979 - val_accuracy: 0.4791 - val_loss: 1.0375
Epoch 7/50
141/141 ━━━━━━━━━━━━━━━━━━━━ 14s 97ms/step - accuracy: 0.4748 - loss: 1.0499 - val_accuracy: 0.4622 - val_loss: 1.0199
Epoch 8/50
141/141 ━━━━━━━━━━━━━━━━━━━━ 15s 98ms/step - accuracy: 0.4647 - loss: 1.0632

Display model accuracy

In [6]:
# Print final training and validation accuracy after training
train_acc = history.history['accuracy'][-1]
val_acc = history.history['val_accuracy'][-1]

print(f"Final training accuracy: {train_acc}")
print(f"Final validation accuracy: {val_acc}")


Final training accuracy: 0.9074527025222778
Final validation accuracy: 0.8575245141983032


Save Model

In [11]:
model_path = '/content/drive/MyDrive/brain_tumor_classifier/brain_tumor_model.h5'
model.save(model_path)